<a href="https://colab.research.google.com/github/cvonholly/technology_complexity/blob/main/add_patent_year.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import numpy as np
import pandas as pd
!pip install python-epo-ops-client
import epo_ops
import xml.etree.ElementTree as ET

In [48]:
# Mount Google Drive files. No need to rerun after running the first time

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [49]:
df = pd.read_excel(r'/content/drive/My Drive/data/labeled_data/LeadAcid_random_1.xlsx')

In [50]:
df.head()

,appln_id,docdb_family_id,publn_nr,publn_auth,earliest_publn_date,appln_title,nb_citing_docdb_fam,tech,title,type,text,class,type.1,comment
0,16880604,8207259,537373,EP,1993-04-21,"High capacity colloidal storage battery, a col...",4,Lead acid,"High capacity colloidal storage battery, a col...",Abstract,The invention provided a new type of high capa...,product,apparatus,NaN
1,16880604,8207259,537373,EP,1993-04-21,"High capacity colloidal storage battery, a col...",4,Lead acid,"High capacity colloidal storage battery, a col...",Claim 1,A high capacity colloidal storage battery whic...,product,apparatus,NaN
2,16880604,8207259,537373,EP,1993-04-21,"High capacity colloidal storage battery, a col...",4,Lead acid,"High capacity colloidal storage battery, a col...",Claim 2,A high capacity colloidal storage battery acco...,product,composition,NaN
3,16880604,8207259,537373,EP,1993-04-21,"High capacity colloidal storage battery, a col...",4,Lead acid,"High capacity colloidal storage battery, a col...",Claim 3,A high capacity colloidal storage battery acco...,product,apparatus,NaN
4,16880604,8207259,537373,EP,1993-04-21,"High capacity colloidal storage battery, a col...",4,Lead acid,"High capacity colloidal storage battery, a col...",Claim 4,A high capacity colloidal storage battery acco...,product,composition,NaN


In [51]:
pub_numbers = df[df['publn_auth']=='EP']['publn_nr'].astype(str).unique()
pub_numbers

array(['537373', '374894', '467861', '820112', '65348', '85109', '669670',
       '1447865', '2204866', '2819206', '2814088', '2811546', '2728650',
       '2897207', '2792534', '2124274'], dtype=object)

In [52]:
consumer_key = "5wpmow69fw4pjAXRrSKKVrlpHpuGgX9bFwlrVmX9XGAKKCHo"
consumer_secret = ""  # insert your own

In [53]:
def resp_to_pub_year(response):
  # Parse XML
  xml_response = response.text

  ns = {"epo": "http://www.epo.org/exchange"}  # Define the namespace
  root = ET.fromstring(xml_response)

  # Find the publication date using XPath
  pub_date = root.find(".//epo:publication-reference/epo:document-id/epo:date", ns)

  if pub_date is not None:
    publication_year = pub_date.text[:4]  # Extract only the year (YYYYMMDD format)
    print(f"Publication Year: {publication_year}")
    return publication_year
  else:
    print("Publication date not found.")
    return None

In [54]:
def get_EP_pub_date(pub_numbers):
  years = []
  client = epo_ops.Client(key=consumer_key, secret=consumer_secret)  # Instantiate client
  for p in pub_numbers:
    p = 'EP' + p
    try:
      response = client.published_data(  # Retrieve bibliography data
        reference_type = 'publication',  # publication, application, priority
        input = epo_ops.models.Epodoc(p),  # epodoc format
        # input = epo_ops.models.Docdb(p, 'EP', 'B1'),  # docdb format
        endpoint = 'biblio',  # optional, defaults to biblio in case of published_data
        constituents = []  # optional, list of constituents
      )
      y = resp_to_pub_year(response)
      if y is not None:
        years.append(y)
      else:
        years.append(np.nan)
    except Exception as e:
      print('client call failed: ')
      print(e)
      years.append(np.nan)
      continue
  return years

In [55]:
years = get_EP_pub_date(pub_numbers)

client call failed: 
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/epodoc/biblio
client call failed: 
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/epodoc/biblio
client call failed: 
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/epodoc/biblio
client call failed: 
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/epodoc/biblio
client call failed: 
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/epodoc/biblio
client call failed: 
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/epodoc/biblio
client call failed: 
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/epodoc/biblio
Publication Year: 2004
Publication Year: 